## XGB Experiments

### Load Dataset

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
import optuna
from optuna.samplers import TPESampler
import joblib
from xgboost import XGBClassifier

In [4]:
random.seed(42)
np.random.seed(42)

- Load the train and test files 

In [6]:
train_df = pd.read_csv("train_data.csv")
val_df  = pd.read_csv("val_data.csv")
test_df  = pd.read_csv("test_data.csv")

In [7]:
print(f"Train: {len(train_df):>6} rows")
print(f" Val : {len(val_df):>6} rows")
print(f" Test: {len(test_df):>6} rows")

Train:   8000 rows
 Val :   1000 rows
 Test:   1000 rows


### TF-IDF - Experiment 1 

In [9]:
def tfidf(train_texts, val_texts):
    vec = TfidfVectorizer()        #tokenization and vectorization
    X_train = vec.fit_transform(train_texts)
    X_val   = vec.transform(val_texts)
    return X_train, X_val

In [10]:
train_texts = train_df["text"].astype(str)
val_texts   = val_df["text"].astype(str)
y_train     = train_df["sentiment_id"]
y_val       = val_df["sentiment_id"]

- Hyperparameter tuning (TF-IDF and logistic regresssion) 

In [12]:
def objective(trial):
    # TF–IDF hyperparameters
    max_df       = trial.suggest_float("max_df", 0.5, 1.0)
    min_df       = trial.suggest_int("min_df", 1, 5)
    ngram_max    = trial.suggest_int("ngram_max", 1, 2)
    max_features = trial.suggest_int("max_features", 1_000, 20_000, step=1_000)

    vectorizer = TfidfVectorizer(
        max_df=max_df,
        min_df=min_df,
        ngram_range=(1, ngram_max),
        max_features=max_features
    )
    X_tr = vectorizer.fit_transform(train_texts)
    X_va = vectorizer.transform(val_texts)

    # XGB hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 10)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3)
    subsample = trial.suggest_float("subsample", 0.6, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.6, 1.0)
    reg_alpha = trial.suggest_float("reg_alpha", 0.0, 1.0)
    reg_lambda = trial.suggest_float("reg_lambda", 0.0, 1.0)

    clf = XGBClassifier(
        objective="multi:softmax",
        num_class=3,
        eval_metric="mlogloss",
        random_state=42,
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda
    )
    
    clf.fit(X_tr, y_train)
    preds = clf.predict(X_va)

    return f1_score(y_val, preds, average="macro")

In [ ]:
sampler = TPESampler(seed=42)
study   = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=10)

print("Best macro-F1:", study.best_value)
print("Best hyperparameters:")
for k, v in study.best_params.items():
    print(f"  {k}: {v}")

[I 2025-07-05 02:11:17,320] A new study created in memory with name: no-name-5c17cf07-685d-49b9-84a4-41cb0e56ae6b
[I 2025-07-05 02:12:11,449] Trial 0 finished with value: 0.5011973799369093 and parameters: {'max_df': 0.6872700594236812, 'min_df': 5, 'ngram_max': 2, 'max_features': 12000, 'n_estimators': 89, 'max_depth': 4, 'learning_rate': 0.026844247528777843, 'subsample': 0.9464704583099741, 'colsample_bytree': 0.8404460046972835, 'reg_alpha': 0.7080725777960455, 'reg_lambda': 0.020584494295802447}. Best is trial 0 with value: 0.5011973799369093.
[I 2025-07-05 02:12:37,523] Trial 1 finished with value: 0.6589270442058881 and parameters: {'max_df': 0.9849549260809971, 'min_df': 5, 'ngram_max': 1, 'max_features': 4000, 'n_estimators': 96, 'max_depth': 5, 'learning_rate': 0.16217936517334897, 'subsample': 0.7727780074568463, 'colsample_bytree': 0.7164916560792167, 'reg_alpha': 0.6118528947223795, 'reg_lambda': 0.13949386065204183}. Best is trial 1 with value: 0.6589270442058881.
[I 2025

- Applying Optimal Parameters for Retraining and Evaluation on the Test Set

In [ ]:
best = study.best_params

pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_df       = best["max_df"],
        min_df       = best["min_df"],
        ngram_range  = (1, best["ngram_max"]),
        max_features = best["max_features"],
    )),
    ("clf", XGBClassifier(
        objective          = "multi:softmax",
        num_class          = 3,
        eval_metric        = "mlogloss",
        random_state       = 42,
        n_estimators       = best["n_estimators"],
        max_depth          = best["max_depth"],
        learning_rate      = best["learning_rate"],
        subsample          = best["subsample"],
        colsample_bytree   = best["colsample_bytree"]
    ))
])

In [ ]:
pipeline.fit(train_texts, y_train)

In [ ]:
y_pred = pipeline.predict(test_df["text"].astype(str))
print(classification_report(test_df["sentiment_id"],y_pred,target_names=["Negative", "Neutral", "Positive"]))

In [ ]:
# Plot the confusion matrix 
cm = confusion_matrix(test_df["sentiment_id"], y_pred)
labels = ["Negative", "Neutral", "Positive"]

fig, ax = plt.subplots()
im = ax.imshow(cm, cmap="Blues")        
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)  
ax.set_xticks(np.arange(len(labels)))
ax.set_xticklabels(labels)
ax.set_yticks(np.arange(len(labels)))
ax.set_yticklabels(labels)
ax.set_xlabel("Predicted Label")
ax.set_ylabel("True Label")
ax.set_title("Confusion Matrix for XGB (TD-IDF1")

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], ha="center", va="center")

plt.tight_layout()
plt.show()

In [ ]:
joblib.dump(pipeline, "xgb_tfidf_experiment1.pkl")